# Variables

## Undeclared Variables

Within a `def` function or REPL environment, you can create a variable with just a name and a value

```python
name = "Sam"
```

A variable declared without `var` or `let` is mutable (it defaults to `var`)

## Mutable `var` and Immutable `let` Variables

In [15]:
var name = "Sam" # Mutable
let user_id = 42 # Immutable

## Type Annotations

Mojo supports dynamic variable types, which can be infered at runtime, and also supports static type annotations on variables enabling strong compile-time type checking providing more predictable, managable and secure code

# Structs

A Mojo struct is a data structure that allows you to encapsulate fields and methods that operate on an abstraction, such as a data type or an object.

For the most part, Mojo’s struct format is designed to provide a static, memory-safe data structure for high-level data types used in programs. For example, all the data types in Mojo’s standard library (such as `Int`, `Bool`, `String`, and `Tuple`) are defined as structs.

## Struct Definition

Notice that the first argument in the `__init__()` method is `inout self`, it’s an argument convention that declares self is a mutable reference. 

### Methods

Methods that take the implicit `self` argument are called instance methods because they act on an instance of the struct.

In [18]:
struct MyPair:
  var first: Int
  var second: Int

  fn __init__(inout self, first: Int, second: Int):
    self.first = first
    self.second = second

  fn get_sum(self) -> Int:
    return self.first + self.second

In [19]:
let mine = MyPair(2, 4)
print(mine.first)
print(mine.get_sum())

2
6


#### Static Methods

A static method can be called without creating an instance of the struct. Unlike instance methods, a static method doesn’t receive the implicit `self` argument, so it can’t access any fields on the struct.

In [22]:
struct Logger:
  fn __init__(inout self):
    pass
  
  @staticmethod
  fn log_info(message: String):
    print("[INFO]: ", message)

In [23]:
Logger.log_info("Static method called.")
var l = Logger()
l.log_info("Static method called from instance.")

[INFO]:  Static method called.
[INFO]:  Static method called from instance.


### Structs Compared to Classes

Eventually, Mojo will also support classes to match the behavior of Python classes.

- Python classes are dynamic: they allow for dynamic dispatch, monkey-patching (or “swizzling”), and dynamically binding instance fields at runtime.

- Mojo structs are static: they are bound at compile-time (you cannot add methods at runtime). Structs allow you to trade flexibility for performance while being safe and easy to use.

- Mojo structs do not support inheritance (“sub-classing”), but a struct can implement *traits*.

- Python classes support class attributes—values that are shared by all instances of the class, equivalent to class variables or static data members in other languages.

- Mojo structs don’t support static data members.

### `@value` Decorator

When you add the `@value` decorator to a struct, Mojo will synthesize the essential lifecycle methods so your object provides full value semantics. Specifically, it generates the `__init__()`, `__copyinit__()`, and `__moveinit__()` methods, which allow you to construct, copy, and move your struct type in a manner that’s value semantic and compatible with Mojo’s ownership model.

When you add the `@value decorator`, Mojo synthesizes each special method above only if it doesn’t exist already. That is, you can still implement a custom version of each method.

In [24]:
@value
struct MyPet:
    var name: String
    var age: Int

Without the copy and move constructors, the following code would not work because Mojo would not know how to copy an instance of `MyPet`:

In [25]:
let dog = MyPet("Charlie", 5)
let poodle = dog
print(poodle.name)

Charlie


# Traits

A *trait* is a set of requirements that a type must implement. You can think of it as a contract: a type that *conforms* to a trait guarantees that it implements all of the features of the trait.

Traits are similar to Java *interfaces*, C++ *concepts*, Swift *protocols*, and Rust *traits*. If you’re familiar with any of those features, Mojo traits solve the same basic problem.

### Traits Can Require Static Methods

In addition to regular instance methods, traits can specify required static methods:

```mojo
trait HasStaticMethod:
    @staticmethod
    fn do_stuff(): ...

fn fun_with_traits[T: HasStaticMethod]():
    T.do_stuff()
```

## Using Traits

Traits solve the problem of writing function overloads for each time you want to have some behavior by letting you define a shared set of behaviors that types can implement. Then you can write a function that depends on the trait, rather than individual types.

In [27]:
trait Quackable:
  fn quack(self):
    pass

@value
struct Duck(Quackable):
  fn quack(self):
    print("Quack")

@value
struct StealthCow(Quackable):
  fn quack(self):
    print("Moo!")

let duck = Duck()
duck.quack()

let cow = StealthCow()
cow.quack()

Quack
Moo!


You can define a function that takes a `Quackable` like this:

In [28]:
fn make_it_quack[T: Quackable](maybe_a_duck: T):
  maybe_a_duck.quack()

In [30]:
make_it_quack(Duck())
make_it_quack(StealthCow())

Quack
Moo!


## Trait Inheritance

Traits can inherit from other traits. A trait that inherits from another trait includes all of the requirements declared by the parent trait

Since `Bird` inherits from `Animal`, a struct that conforms to the `Bird` trait needs to implement both `make_sound()` and `fly()`. And since every `Bird` conforms to `Animal`, a struct that conforms to `Bird` can be passed to any function that requires an `Animal`.

In [32]:
trait Animal:
  fn make_sound(self):
    pass

# Bird inherits from Animal
trait Bird(Animal):
  fn fly(self):
    pass

# Functions

Functions can be declared with `fn` or `def`

### Value ownership and argument mutability

`def` functions receive arguments "by value"

`fn` functions receive arguments "by immutable reference"

Mojo allows you to specify for each argument whether it should be passed by value (as `owned`), or whether it should be passed by reference (as `borrowed` for an immutable reference, or as `inout` for a mutable reference)

See below section on "Value Ownership"

In [3]:
fn greet(name: String) -> String:
  return "Hello, " + name

In [6]:
greeting = greet("Charlie Roth")
print(greeting)

Hello, Charlie Roth


# Value Ownership

## Introduction to Value Semantics

In the most basic situation, sharing a value-semantic type means that you create a copy of the value. This is also known as “pass by value.”

In [7]:
x = 1
y = x
y += 1
print(x)
print(y)

1
2


Each variable has exclusive ownership of a value

If a type instead uses reference semantics, then y would point to the same value as x, and incrementing either one would affect the value for both. Neither x nor y would “own” the value, and any variable would be allowed to reference it and mutate it

Another example

In [8]:
def add_one(y: Int):
  y += 1
  print(y)

x = 1
add_one(x)
print(x)

2
1


If you’re familiar with Python, this is probably familiar so far, because the code above behaves the same in Python. However, Python is not value semantic. It gets complicated, but let’s consider a situation in which you call a Python function and pass an object with a pointer to a heap-allocated value. Python actually gives that function a reference to your object, which allows the function to mutate the heap-allocated value. This can cause nasty bugs if you’re not careful, because the function might incorrectly assume it has unique ownership of that object.

In Mojo, the default behavior for all function arguments is to use value semantics. If the function wants to modify the value of an incoming argument, then it must explicitly declare so, which avoids accidental mutations.

For example, even though the Mojo `Tensor` type allocates values on the heap, when you pass an instance to a def function, it creates a unique copy of all values. Thus, if we modify the argument in the function, the original value is unchanged:

In [10]:
def update_tensor(t: Tensor[DType.uint8]):
  t[1] = 3
  print(t)

In [11]:
t = Tensor[DType.uint8](2)
t[0] = 1
t[1] = 2
update_tensor(t)
print(t)

Tensor([[1, 3]], dtype=uint8, shape=2)
Tensor([[1, 2]], dtype=uint8, shape=2)


If this were Python code, the function would modify the original object, because Python shares a reference to the original object.

### Values Semantics in `def` vs `fn`

The arguments above are mutable because a `def` function gets ownership for its arguments by default (usually as a copy). Whereas, `fn` functions instead receive arguments as immutable references, by default. This is a memory optimization to avoid making unnecessary copies.

In [13]:
fn add_two(y: Int):
    # y += 2 # This will cause a compiler error because `y` is immutable
    # We can instead make an explicit copy:
    var z = y
    z += 2
    print(z)

x = 1
add_two(x)
print(x)

3
1


The way the fn function receives the `y` value is a “look but don’t touch” approach to value semantics. This is also a more memory-efficient approach when dealing with memory-intensive arguments, because Mojo doesn’t make any copies unless we explicitly make the copies ourselves.

# Ownership and Borrowing

When using some programming languages is that you must manually allocate and deallocate memory. When multiple parts of the program need access to the same memory, it becomes difficult to keep track of who “owns” a value and determine when is the right time to deallocate it. If you make a mistake, it can result in a “use-after-free” error, a “double free” error, or a “leaked memory” error, any one of which can be catastrophic.

Mojo helps avoid these errors by ensuring there is only one variable that owns each value at a time, while still allowing you to share references with other functions. When the lifetime of the owner ends, Mojo destroys the value.

## Argument Conventions

An argument convention specifies whether an argument is mutable or immutable, and whether the function owns the value. Each convention is defined by a keyword at the beginning of an argument declaration:

- `borrowed`: The function receives an immutable reference. This means the function can read the original value (it is not a copy), but it cannot mutate (modify) it.

- `inout`: The function receives a mutable reference. This means the function can read and mutate the original value (it is not a copy).

- `owned`: The function takes ownership. This means the function has exclusive mutable access to the argument—the function caller does not have access to this value (anymore). Often, this also implies that the caller should transfer ownership to this function, but that’s not always what happens and this might instead be a copy (as you’ll learn below).

In [14]:
fn add(inout x: Int, borrowed y: Int):
  x += y

var a = 1
let b = 2
add(a, b)
print(a)

3
